In [2]:
from neo4j import GraphDatabase
import pandas as pd

In [3]:
#Connection

database_connection = GraphDatabase.driver(uri = "bolt://localhost:7687", auth=("neo4j", "neo5j"))
session = database_connection.session()  

In [4]:
#Separando los resultados por variable

def split_variables(resobj):
    varlist = []
    keys = resobj.keys()
    data = resobj.data()
    for idx, var in enumerate(keys):
        #vardict = { keys[kidx] : var for kidx, var in enumerate(data) }
        attriblist = [dct[var] for dct in data]
        varlist.append(attriblist)
        
    vardict = { keys[kidx] : val for kidx, val in enumerate(varlist) }
    return vardict

## Funciones de consulta

In [157]:
def consulta1(producto,estado,num_resultados):
    s1 = "MATCH (p:productos {Tipo_Producto :'"+producto+"'}) "
    s2 = "MATCH (e:estados {Nombre_Estado:'"+estado+"'}) "
    s3 = "MATCH (t:tiendas) \
MATCH (e) -[v]- (t) -- (p) \
RETURN t"

    c1 = s1+s2+s3
    result = session.run(c1)
    res = split_variables(result)
    if len(res['t']) > num_resultados:
        #print(res['t'][:num_resultados])
        return list(map((lambda dct : dct['Nombre_Tienda']), res['t'][:num_resultados]))
    else:
        #print(res['t'])
        return list(map((lambda dct : dct['Nombre_Tienda']), res['t']))   

In [158]:
consulta1('BRANDY','AGUASCALIENTES',5)

['TIENDAS SORIANA  S.A. DE C.V.',
 'SUSHITO - JUAN FERNANDO OROZCO CARDONA',
 'RESTAURANTE AUGUSTOÃ¯S S.A. DE C.V.',
 'WINGS AIR FORCE- FRANQUICIAS WINGS AIR FORCE  S. DE R.L. DE C.V.',
 'LA MANSION- RICHYCARLS  S.A. DE C.V.']

In [145]:
def consulta2(estado,tienda):
    s1 = "MATCH (e:estados {Nombre_Estado :'"+estado+"'}) "
    s2 = "MATCH (t:tiendas {Nombre_Tienda: '"+tienda+"'}) "
    s3 = "MATCH (p:productos) \
MATCH (e) -- (t) -[v]- (p) \
RETURN e, t, p, v.incumplimiento"
    c2 = s1+s2+s3

    result = session.run(c2)
    res = split_variables(result)
    return res['v.incumplimiento']

In [146]:
consulta2('BAJA CALIFORNIA','CARNE MART SUCURSAL TECATE')

['NO SE DETECTO INCUMPLIMIENTO']

In [100]:
c0 = 'match (n) RETURN n'

result = session.run(c0)
res = split_variables(result)
res['n'][:20]

In [132]:
c1

"MATCH (p:productos {Tipo_Producto:'BRANDY'}) MATCH (e:estados {Nombre_Estado:'AGUASCALIENTES'}) MATCH (t:tiendas) MATCH (e) -[v]- (t) -- (p) RETURN t"

In [119]:
# dado estado y producto   buscar lugares donde se pueda encontrar

c1 = "MATCH (p:productos {Tipo_Producto:'BRANDY'}) \
MATCH (e:estados {Nombre_Estado:'AGUASCALIENTES'}) \
MATCH (t:tiendas) \
MATCH (e) -[v]- (t) -- (p) \
RETURN t"

result = session.run(c1)
res = split_variables(result)
res['t'][:5]

[{'Id_Tienda': '137t', 'Nombre_Tienda': 'TIENDAS SORIANA  S.A. DE C.V.'},
 {'Id_Tienda': '82t',
  'Nombre_Tienda': 'SUSHITO - JUAN FERNANDO OROZCO CARDONA'},
 {'Id_Tienda': '74t', 'Nombre_Tienda': 'RESTAURANTE AUGUSTOÃ¯S S.A. DE C.V.'},
 {'Id_Tienda': '66t',
  'Nombre_Tienda': 'WINGS AIR FORCE- FRANQUICIAS WINGS AIR FORCE  S. DE R.L. DE C.V.'},
 {'Id_Tienda': '65t', 'Nombre_Tienda': 'LA MANSION- RICHYCARLS  S.A. DE C.V.'}]

In [81]:
#dado estado y tienda ver si tiene incumplimientos.

c2 = "MATCH (e:estados {Nombre_Estado :'BAJA CALIFORNIA'}) \
MATCH (t:tiendas {Nombre_Tienda: 'CARNE MART SUCURSAL TECATE'}) \
MATCH (p:productos) \
MATCH (e) -- (t) -[v]- (p) \
RETURN e, t, p, v.incumplimiento"

result = session.run(c2)
res = split_variables(result)
res

In [144]:
# Dado un estado y un producto. buscar alternativas sin incumplimiento de ese producto

c3 = "MATCH (e:estados {Nombre_Estado :'BAJA CALIFORNIA'}) \
MATCH (p:productos {Tipo_Producto:'BRANDY'}) \
MATCH (t:tiendas) \
MATCH (e) -- (t) -[v {incumplimiento : 'NO SE DETECTO INCUMPLIMIENTO'}]- (p) \
RETURN e, p, t, v.incumplimiento"

result = session.run(c3)
res = split_variables(result)
res

{'e': [{'Nombre_Estado': 'BAJA CALIFORNIA', 'Id_Estado': 'BAJA CALIFORNIA'},
  {'Nombre_Estado': 'BAJA CALIFORNIA', 'Id_Estado': 'BAJA CALIFORNIA'}],
 'p': [{'Id_Producto': '2p', 'Tipo_Producto': 'BRANDY'},
  {'Id_Producto': '2p', 'Tipo_Producto': 'BRANDY'}],
 't': [{'Id_Tienda': '137t', 'Nombre_Tienda': 'TIENDAS SORIANA  S.A. DE C.V.'},
  {'Id_Tienda': '1692t', 'Nombre_Tienda': '7 ELEVEN MEXICO  S.A. DE C.V.'}],
 'v.incumplimiento': ['NO SE DETECTO INCUMPLIMIENTO',
  'NO SE DETECTO INCUMPLIMIENTO']}

In [ ]:
c4 = "()-[tp: vende]->() \
      RETURN t.Nombre_Tienda, \
      p.Nombre_Producto \
      LIMIT 10"

In [195]:
result = session.run(c4)
result
res = split_variables(result)
res

CypherSyntaxError: {code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input '(': expected <init> (line 1, column 1 (offset: 0))
"()-[tp: vende]->()       RETURN t.Nombre_Tienda,       p.Nombre_Producto       LIMIT 10"
 ^}

In [196]:
c5 = "MATCH (t:Tienda) -> \
      [te:localizada_en] -> \
      (e.Estado) \
      CALL {with store \
      MATCH (t:Tienda) - \
      [tp: vende] -> \
      (p:Producto) \
      WHERE tp.INCUMPLIMIENTO = 'NO SE DETECTO INCUMPLIMIENTO' \
      RETURN count(p) as X} \
      RETURN e.Nombre_Estado, \
      sum(X) as Cantidad_Productos_Sin_Incumplimiento, \
      count(Tienda) as Cantidad_Tiendas, \
      toFloat(sum(X)/toFloat(count(Tienda))) as Ratio \
      ORDER BY Ratio"

c5[:20]

'MATCH (t:Tienda) -> '

In [197]:
result = session.run(c5)
result
res = split_variables(result)
res

CypherSyntaxError: {code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input '-': expected
  ","
  "CALL"
  "CREATE"
  "DELETE"
  "DETACH"
  "FOREACH"
  "LOAD"
  "MATCH"
  "MERGE"
  "OPTIONAL"
  "REMOVE"
  "RETURN"
  "SET"
  "UNION"
  "UNWIND"
  "USE"
  "USING"
  "WHERE"
  "WITH"
  <EOF> (line 1, column 18 (offset: 17))
"MATCH (t:Tienda) ->       [te:localizada_en] ->       (e.Estado)       CALL {with store       MATCH (t:Tienda) -       [tp: vende] ->       (p:Producto)       WHERE tp.INCUMPLIMIENTO = 'NO SE DETECTO INCUMPLIMIENTO'       RETURN count(p) as X}       RETURN e.Nombre_Estado,       sum(X) as Cantidad_Productos_Sin_Incumplimiento,       count(Tienda) as Cantidad_Tiendas,       toFloat(sum(X)/toFloat(count(Tienda))) as Ratio       ORDER BY Ratio"
                  ^}